# The Python Database API Specification

PEP 249 defines the Python Database API Specification v2.0: https://www.python.org/dev/peps/pep-0249/


# Python SQLite
"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.

The sqlite3 module was written by Gerhard Häring. It provides a SQL interface compliant with the DB-API 2.0 specification described by PEP 249." https://docs.python.org/3/library/sqlite3.html
The official SQLite documentation can be found at https://www.sqlite.org/index.html.

A lightweight and platform-independent GUI tool for SQLite databsaes is the *DB Browser for SQLite* that can be downloaded at https://sqlitebrowser.org/.

A great tutorial of how to get started with SQLite in Python can be found at https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto.

## Create / connect to a SQLite database

```python
import sqlite3

# DB file
sqlite_file = 'my_db.sqlite' 

# connect to the DB and create a cursor object
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# do DB operations here...

# close the connection to the DB
conn.close()
```

https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto#connecting-to-an-sqlite-database

## Create new tables

```python
import sqlite3

sqlite_file = 'my_db.sqlite'
table_name1 = 'my_table_1'
table_name2 = 'my_table_2'
column1 = 'my_1st_column'
field_type = 'INTEGER'

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Creating a new SQLite table with 1 column
c.execute('CREATE TABLE {tn} ({c1} {ft})'.format(tn=table_name1, c1=column1, ft=field_type))

# Creating a second table with 1 column and set it as PRIMARY KEY
# note that PRIMARY KEY column must consist of unique values!
c.execute('CREATE TABLE {tn} ({c1} {ft} PRIMARY KEY)'.format(tn=table_name2, c1=column1, ft=field_type))

# Committing changes and closing the connection to the database file
conn.commit()
conn.close()
```

https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto#creating-a-new-sqlite-database

## Adding new columns

```python
import sqlite3

sqlite_file = 'my_db.sqlite'
table_name = 'my_table_2'
column2 = 'my_2nd_column'
column3 = 'my_3nd_column'
column_type = 'TEXT'
default_val = 'Hello World'

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# A) Adding a new column without a row value
c.execute("ALTER TABLE {tn} ADD COLUMN '{c2}' {ct}".format(tn=table_name, c2=column2, ct=column_type))

# B) Adding a new column with a default row value
c.execute("ALTER TABLE {tn} ADD COLUMN '{c3}' {ct} DEFAULT '{df}'".format(tn=table_name, c3=column3, ct=column_type, df=default_val))

# Committing changes and closing the connection to the database file
conn.commit()
conn.close()
```

https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto#adding-new-columns

## Insert or update data

```python
import sqlite3

sqlite_file = 'my_db.sqlite'
table_name = 'my_table_2'
column1 = 'my_1st_column'
column2 = 'my_2nd_column'

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# A) Inserts an ID with a specific value in a second column 
try:
    c.execute("INSERT INTO {tn} ({c1}, {c2}) VALUES (123456, 'test')".format(tn=table_name, c1=column1, c2=column2))
except sqlite3.IntegrityError:
    print('ERROR: ID already exists in PRIMARY KEY column {}'.format(column1))

# B) Tries to insert an ID (if it does not exist yet)
# with a specific value in a second column 
c.execute("INSERT OR IGNORE INTO {tn} ({c1}, {c2}) VALUES (123456, 'test')".format(tn=table_name, c1=column1, c2=column2))

# C) Updates the newly inserted or pre-existing entry            
c.execute("UPDATE {tn} SET {c2}=('Hi World') WHERE {c1}=(123456)".format(tn=table_name, c2=column2, c1=column1))

conn.commit()
conn.close()
```

https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto#inserting-and-updating-rows

## Query data

```python
import sqlite3

sqlite_file = 'my_db.sqlite'
table_name = 'my_table_2'
column1 = 'my_1st_column'
some_id = 123456
column2 = 'my_2nd_column'

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# 1) Contents of all columns for row that match a certain value in 1 column
c.execute('SELECT * FROM {tn} WHERE {c2}="Hi World"'.format(tn=table_name, c2=column2))
all_rows = c.fetchall()
print('1):', all_rows)

# 2) Value of a particular column for rows that match a certain value in column2 
c.execute('SELECT ({coi}) FROM {tn} WHERE {c2}="Hi World"'.format(coi=column2, tn=table_name, c2=column2))
all_rows = c.fetchall()
print('2):', all_rows)

# 3) Value of 2 particular columns for rows that match a certain value in 1 column
c.execute('SELECT {c2},{c1} FROM {tn} WHERE {c2}="Hi World"'.format(c2=column2, c1=column1, tn=table_name))
all_rows = c.fetchall()
print('3):', all_rows)

# 4) Selecting only up to 10 rows that match a certain value in 1 column
c.execute('SELECT * FROM {tn} WHERE {c2}="Hi World" LIMIT 10'.format(tn=table_name, c2=column2))
ten_rows = c.fetchall()
print('4):', ten_rows)

# 5) Check if a certain ID exists and print its column contents
c.execute("SELECT * FROM {tn} WHERE {c1}={my_id}".format(tn=table_name, c1=column1, my_id=some_id))
id_exists = c.fetchone()
if id_exists:
    print('5): {}'.format(id_exists))
else:
    print('5): {} does not exist'.format(some_id))

# Closing the connection to the database file
conn.close()
```

https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto#querying-the-database---selecting-rows

## Data types in SQLite

SQLite natively supports the following types: ```NULL, INTEGER, REAL, TEXT, BLOB```.

Sending Python types to SQLite:

| Python type | SQLite type |
|:------------|-------------|
| None        | NULL        |
| int         | INTEGER     |
| float       | REAL        |
| str         | TEXT        |
| bytes       | BLOB        |

Converting SQLite types to Python types:

| SQLite type | Python type                                         |
|:------------|-----------------------------------------------------|
| NULL        | None                                                |
| INTEGER     | int                                                 |
| REAL        | float                                               |
| TEXT        | depends on ```test_factory```, ```str``` by default |
| BLOB        | bytes                                               |


https://docs.python.org/3/library/sqlite3.html#introduction

https://www.sqlite.org/datatype3.html

## Adapters and Converters

"The type system of the sqlite3 module is extensible in two ways: you can store additional Python types in a SQLite database via **object adaptation**, and you can let the sqlite3 module convert SQLite types to different Python types via **converters**." https://docs.python.org/3/library/sqlite3.html#introduction


https://docs.python.org/3/library/sqlite3.html#using-adapters-to-store-additional-python-types-in-sqlite-databases describes how to store additional Python types in SQLite and https://docs.python.org/3/library/sqlite3.html#converting-sqlite-values-to-custom-python-types explains how to convert SQLite values to custom Python types.

# SQLite Exercise

In [4]:
# Creating a new SQLite database

import sqlite3
conn = sqlite3.connect('my_db.sqlite')
c = conn.cursor()

In [5]:
table_name1 = 'my_table_1'
table_name2 = 'my_table_2'
column1 = 'my_1st_column'
field_type = 'INTEGER'

# Creating a new SQLite table with 1 column
# Creating a second table with 1 column and set it as PRIMARY KEY
# note that PRIMARY KEY column must consist of unique values!
c.execute('CREATE TABLE {tn} ({c1} {ft} PRIMARY KEY)'.format(tn=table_name2, c1=column1, ft=field_type))

# Committing changes and closing the connection to the database file
conn.commit()


In [6]:
# Adding new columns
table_name = 'my_table_2'
id_column = 'my_1st_column'
column2 = 'my_2nd_column'
column3 = 'my_3nd_column'
column_type = 'TEXT'
default_val = 'Hello World'

# A) Adding a new column without a row value
c.execute("ALTER TABLE {tn} ADD COLUMN '{c2}' {ct}".format(tn=table_name, c2=column2, ct=column_type))

# B) Adding a new column with a default row value
c.execute("ALTER TABLE {tn} ADD COLUMN '{c3}' {ct} DEFAULT '{df}'".format(tn=table_name, c3=column3, ct=column_type, df=default_val))

# Committing changes and closing the connection to the database file
conn.commit()


In [7]:
# Inserting and updating rows
table_name = 'my_table_2'
column1 = 'my_1st_column'
column2 = 'my_2nd_column'

# A) Inserts an ID with a specific value in a second column 
try:
    c.execute("INSERT INTO {tn} ({c1}, {c2}) VALUES (123456, 'test')".format(tn=table_name, c1=column1, c2=column2))
except sqlite3.IntegrityError:
    print('ERROR: ID already exists in PRIMARY KEY column {}'.format(id_column))

# B) Tries to insert an ID (if it does not exist yet)
# with a specific value in a second column 
c.execute("INSERT OR IGNORE INTO {tn} ({c1}, {c2}) VALUES (123456, 'test')".format(tn=table_name, c1=column1, c2=column2))

# C) Updates the newly inserted or pre-existing entry            
c.execute("UPDATE {tn} SET {c2}=('Hi World') WHERE {c1}=(123456)".format(tn=table_name, c2=column2, c1=column1))

conn.commit()

In [8]:
# close the DB connection
conn.close()

# European Election 2019 Exercise
* Import the "eu2019.csv" file into your SQLite database.
* Check the data types. Are the data types correct? If not, change them.
* Check the column names. Rename them to something easy to write queries for (remove whitespace, special characters etc.)
* Are there missing values in the data? What do the missing values indicate? Think about the consequences for calculating some measures such as counting the total number of votes for a party.
* How many different parties are part of the data? Is there a special case in the data (tip: count the number of Kreis Nr. per party)?
* Which city district (Kreis) is the biggest and which is the smallest based on the number of votes?
* Which party got the most number of votes in the city districts? Which district is it?
* Filter the data for the states (Bundesländer) and get the top 3 parties based on the percentage of votes. How much are the percentages?
* Sum the 'Gültige Stimmen %' column for
  * Bundesrepublik Deutschland
  * the states (Bundesländer)
  * the city districts (Kreis) by 'Kreis Nr.' or 'Gebietsbezeichnung' columns  
  Explain the results. Do all add up to ~100%? Why does the group by column 'Kreis Nr.' and 'Gebietsbezeichnung' produce different results for the city districts?

# Python and MySQL (MySQLdb)

"MySQLdb is an interface for connecting to a MySQL database server from Python. It implements the Python Database API v2.0 and is built on top of the MySQL C API."

https://www.tutorialspoint.com/python/python_database_access.htm

# Python and Oracle

A turorial of how to use Python With Oracle Database 11g can be found at https://www.oracle.com/webfolder/technetwork/tutorials/obe/db/OOW11/python_db/python_db.htm.

# Python and ODBC

"In computing, Open Database Connectivity (ODBC) is a standard application programming interface (API) for accessing database management systems (DBMS). The designers of ODBC aimed to make it independent of database systems and operating systems. An application written using ODBC can be ported to other platforms, both on the client and server side, with few changes to the data access code." https://en.wikipedia.org/wiki/Open_Database_Connectivity

"pyodbc is an open source Python module that makes accessing ODBC databases simple. It implements the DB API 2.0 specification but is packed with even more Pythonic convenience." https://github.com/mkleehammer/pyodbc

The wiki at https://github.com/mkleehammer/pyodbc/wiki provides documentation for using pyodbc.

# Python Object Databases (ZODB-Zope Object Database) 

An introduction to ZODB can be found at http://www.zodb.org/en/latest/articles/ZODB1.html.

## Install ZODB
A tutorial of how to install ZODB can be found at http://www.zodb.org/en/latest/tutorial.html#installation.

```bash
$ conda install zodb```

## ACID standards in ZODB
* **Atomicity:** "means that any changes to data made during a transaction are all-or-nothing. Either all the changes are applied, or none of them are. If a program makes a bunch of modifications and then crashes, the database won’t be partially modified, potentially leaving the data in an inconsistent state; instead all the changes will be forgotten. That’s bad, but it’s better than having a partially- applied modification put the database into an inconsistent state."
* **Consistency:** "means that each transaction executes a valid transformation of the database state. Some databases, but not ZODB, provide a variety of consistency checks in the database or language; for example, a relational database constraint columns to be of particular types and can enforce relations across tables. Viewed more generally, atomicity and isolation make it possible for applications to provide consistency."
* **Isolation:** "means that two programs or threads running in two different transactions cannot see each other’s changes until they commit their transactions."
* **Durability:** "means that once a transaction has been committed, a subsequent crash will not cause any data to be lost or corrupted."

http://www.zodb.org/en/latest/articles/old-guide/prog-zodb.html


## Getting started with ZODB
A tutorial of how to get started with ZODB and understanding the main concepts can be found at http://www.zodb.org/en/latest/tutorial.html#tutorial-label.


create storage and use the ‘mydatabse.fs’ file to store the object information. Then, you create a database that uses that storage.

Next, the database needs to be “opened” by calling the ‘open()’ method. This will return a connection object to the database. The connection object then gives you access to the ‘root’ of the database with the ‘root()’ method.

The ‘root’ object is the dictionary that holds all of your persistent objects. For example, you can store a simple list of strings in the root object:

```python
from ZODB import FileStorage, DB
# create storage
storage = FileStorage.FileStorage('mydatabase.fs')
# create a database using the storage
db = DB(storage)
# connect to the database
connection = db.open()
# access the root object
root = connection.root()
```

```python
# store a list of strings in the root object 
root['employees'] = ['Mary', 'Jo', 'Bob']
```

```python
import transaction
# commit the transaction
transaction.commit()
```

```python
# access the items of the root object
root.items()
[('employees', ['Mary', 'Jo', 'Bob'])]
```

```python
# close the connection
connection.close()
```


## Rules of Persistence
"When implementing persistent objects, be aware that an object’s attributes should be :
* immutable (such as strings or integers),
* persistent (subclass Persistent), or
* You need to take special precautions."

http://www.zodb.org/en/latest/guide/writing-persistent-objects.html#rules-of-persistence

# ZODB Exercise

In [1]:
from ZODB import FileStorage, DB
# create storage
storage = FileStorage.FileStorage('mydatabase.fs')
# create a database using the storage
db = DB(storage)
# connect to the database
connection = db.open()
# access the root object
root = connection.root()

In [2]:
# store a list of strings in the root object 
root['employees'] = ['Mary', 'Jo', 'Bob']

In [3]:
import transaction
# commit the transaction
transaction.commit()

In [4]:
# access the items of the root object
root.items()
[('employees', ['Mary', 'Jo', 'Bob'])]

[('employees', ['Mary', 'Jo', 'Bob'])]

In [5]:
# close the connection
connection.close()

# SQLAlchemy
"SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language."

* Python SQL Toolkit (Core): pythonic way to generate SQL queries
* Object Relational Mapper (ORM): DB relations as Python objects


https://www.sqlalchemy.org/

```bash
$ conda install sqlalchemy
```

In [9]:
import sqlalchemy
sqlalchemy.__version__

'1.3.4'

## SQLAlchemy Core Example

In [10]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://')

In [11]:
from sqlalchemy import Column, Integer, Text, MetaData, Table

metadata = MetaData()
messages = Table(
    'messages', metadata,
    Column('id', Integer, primary_key=True),
    Column('message', Text),
)

messages.create(bind=engine)
 

In [12]:
insert_message = messages.insert().values(message='Hello, World!')
engine.execute(insert_message)

In [13]:
from sqlalchemy import select
stmt = select([messages.c.message]) 
message, = engine.execute(stmt).fetchone()
print(message)
 

Hello, World!


## SQLAlchemy ORM Example

In [14]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://')
 

In [15]:
from sqlalchemy.ext.declarative import declarative_base
# define declarative to map classes and tables
Base = declarative_base()

In [16]:
from sqlalchemy import Column, Integer, String

class Message(Base):
    __tablename__ = 'messages'
    
    id = Column(Integer, primary_key=True)
    message = Column(String)
 

In [17]:
# create table and mapper
Base.metadata.create_all(engine)

In [18]:
# insert data (but not saved yet)
message = Message(message="Hello World!")
message.message

'Hello World!'

In [19]:
# create a session to save something to the DB
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
 

In [20]:
# add data to the session and commit the changes
session.add(message)
session.commit()

In [21]:
# query the DB
query = session.query(Message)
instance = query.first()
print (instance.message)

Hello World!
